<a href="https://colab.research.google.com/github/dakilaledesma/herbarium_model_testing/blob/main/2x3xnet/H2022_GPU_DualNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%time
! unzip -q /content/drive/MyDrive/UNC/H2022/RCNN_PP/pls224p56_Orchidaceae_train.zip -d /content/
! unzip -q /content/drive/MyDrive/UNC/H2022/orchidaceae_train.zip -d /content/

CPU times: user 102 ms, sys: 30.8 ms, total: 133 ms
Wall time: 19.5 s


In [2]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Concatenate
import numpy as np

classes = 300

orig_model = ResNet50V2(weights='imagenet', include_top=False)
lg_model = ResNet50V2(weights='imagenet', include_top=False)

for ln in [layer.name for layer in lg_model.layers]:
    lg_model.get_layer(name=ln)._name = f"lg_{ln}"

out = Concatenate()([orig_model.output, lg_model.output])
out = Dense(2048, activation='relu')(out)
out = Dense(classes, activation='softmax', name='predictions')(out)

model = Model(inputs=[orig_model.input, lg_model.input], outputs=out)

model.compile(optimizer='adam', loss='categorical_crossentropy')

94683136/94668760 [==============================] - 2s 0us/step


In [3]:
classes = 300

In [4]:
from glob import glob
import os
cat_to_int = {}
for i, f in enumerate(sorted(glob("pls224p56_Orchidaceae_train/*"))):
  cat = os.path.basename(f)
  cat_to_int[cat] = i

In [5]:
valid_fn = {}
for fn in sorted(glob("pls224p56_Orchidaceae_train/**/*.*", recursive=True)):
  bn = os.path.basename(fn)
  valid_fn[bn] = fn

In [6]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
from tensorflow.keras.preprocessing import image
from tqdm.notebook import tqdm
import numpy as np

X = []
y = []
orc = sorted(glob("orchidaceae_train/**/*.*", recursive=True))
for fn in tqdm(orc, total=len(list(orc))):
  bn =  os.path.basename(fn)
  cat = cat_to_int[fn.split("/")[-2]]
  try:
    pl_fn = valid_fn[bn]
  except KeyError:
    pass

  pair = []
  for f in [fn, pl_fn]:
    img_path = f
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = preprocess_input(x)
    pair.append(x)

  X.append(pair)

  one_hot = np.zeros((classes,), dtype='int')
  one_hot[cat] = 1
  y.append(one_hot)


  0%|          | 0/9419 [00:00<?, ?it/s]

In [7]:
X = np.array(X)

In [8]:
y = np.array(y)

In [9]:
print(X.shape, y.shape)

(9419, 2, 224, 224, 3) (9419, 300)


In [10]:
model.fit(x=[X[:, 0], X[:, 1]], y=y, batch_size=32, epochs=24)

Epoch 1/24


ValueError: ignored